<a href="https://colab.research.google.com/github/MarigoldJ/ygl2/blob/main/class/20210615_nlp_day4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LSA

## LSA 실습하기

In [25]:
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import Normalizer

import pandas as pd
import numpy as np

In [26]:
def get_dtm(sentence):
    '''
    dtm을 만드는 함수
    '''
    vectorizer = CountVectorizer(min_df=1, stop_words='english', dtype=np.float32) # 불용어 한국어 지원 X...
    dtm = vectorizer.fit_transform(sentence)    # dtm : document-term matrix

    return dtm, vectorizer

def print_dtm_matrix(dtm, vectorizer, sentence):
    return pd.DataFrame(dtm.toarray(), index=sentence, columns=vectorizer.get_feature_names())

def lsa_tsvd(n_components, dtm, vectorizer, sentence):
    
    # dtm에 truncated SVD 적용
    lsa = TruncatedSVD(n_components, algorithm='arpack')    # arpack : 기본적인 알고리즘
    dtm_lsa = lsa.fit_transform(dtm)

    # dtm에 normalizer 적용
    dtm_lsa = Normalizer(copy=False).fit_transform(dtm_lsa)

    # 출력
    components = pd.DataFrame(lsa.components_, index=['components-1', 'components-2'], columns=vectorizer.get_feature_names())
    lsa = pd.DataFrame(dtm_lsa, index=sentence, columns=['components-1', 'components-2'])

    # dtm과 dtm_lsa의 유사도 확인
    similarity = np.asarray(np.asmatrix(dtm_lsa) * np.asmatrix(dtm_lsa).T)
    similarity = pd.DataFrame(similarity, index=sentence, columns=sentence)

    return components, lsa, similarity

def main():
    sentences = ['중앙방역대책본부는 오늘 0시 기준 코로나19 확진 환자가 44명 추가 확인돼 모두 13,417명으로 늘었다고 밝혔습니다.', 
                 '신규 확진 환자 가운데 해외 유입 사례는 23명, 국내 발생은 21명입니다.', 
                 '국내 발생 환자는 지역별로 서울 7명, 경기 8명 등 수도권에서 15명이 확인됐고, 광주에서도 5명, 대전에서도 1명이 확진됐습니다.', 
                 '또한 코로나19로 1명이 추가 사망해 누적 사망자는 모두 289명으로 늘었습니다.']
    dtm, vectorizer = get_dtm(sentences)
    d = print_dtm_matrix(dtm, vectorizer, sentences)

    c, l, s = lsa_tsvd(2, dtm, vectorizer, sentences)

    return d, c, l, s


In [27]:
d, c, l, s = main()

In [32]:
d.head(2)

,0시,13,15명이,1명이,21명입니다,23명,289명으로,417명으로,44명,5명,7명,8명,가운데,경기,광주에서도,국내,기준,누적,늘었다고,늘었습니다,대전에서도,또한,모두,발생,발생은,밝혔습니다,사례는,사망자는,사망해,서울,수도권에서,신규,오늘,유입,중앙방역대책본부는,지역별로,추가,코로나19,코로나19로,해외,확인돼,확인됐고,확진,확진됐습니다,환자,환자가,환자는
"중앙방역대책본부는 오늘 0시 기준 코로나19 확진 환자가 44명 추가 확인돼 모두 13,417명으로 늘었다고 밝혔습니다.",1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
"신규 확진 환자 가운데 해외 유입 사례는 23명, 국내 발생은 21명입니다.",0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0


In [29]:
c

,0시,13,15명이,1명이,21명입니다,23명,289명으로,417명으로,44명,5명,7명,8명,가운데,경기,광주에서도,국내,기준,누적,늘었다고,늘었습니다,대전에서도,또한,모두,발생,발생은,밝혔습니다,사례는,사망자는,사망해,서울,수도권에서,신규,오늘,유입,중앙방역대책본부는,지역별로,추가,코로나19,코로나19로,해외,확인돼,확인됐고,확진,확진됐습니다,환자,환자가,환자는
components-1,0.115642,0.115642,0.198653,0.263701,0.056029,0.056029,0.065048,0.115642,0.115642,0.198653,0.198653,0.198653,0.056029,0.198653,0.198653,0.254682,0.115642,0.065048,0.115642,0.065048,0.198653,0.065048,0.180691,0.198653,0.056029,0.115642,0.056029,0.065048,0.065048,0.198653,0.198653,0.056029,0.115642,0.056029,0.115642,0.198653,0.180691,0.115642,0.065048,0.056029,0.115642,0.198653,0.171671,0.198653,0.056029,0.115642,0.198653
components-2,0.205450,0.205450,-0.139691,-0.090792,0.014464,0.014464,0.048898,0.205450,0.205450,-0.139691,-0.139691,-0.139691,0.014464,-0.139691,-0.139691,-0.125227,0.205450,0.048898,0.205450,0.048898,-0.139691,0.048898,0.254349,-0.139691,0.014464,0.205450,0.014464,0.048898,0.048898,-0.139691,-0.139691,0.014464,0.205450,0.014464,0.205450,-0.139691,0.254349,0.205450,0.048898,0.014464,0.205450,-0.139691,0.219914,-0.139691,0.014464,0.205450,-0.139691


In [30]:
l

,components-1,components-2
"중앙방역대책본부는 오늘 0시 기준 코로나19 확진 환자가 44명 추가 확인돼 모두 13,417명으로 늘었다고 밝혔습니다.",0.515354,0.856977
"신규 확진 환자 가운데 해외 유입 사례는 23명, 국내 발생은 21명입니다.",0.972027,0.234870
"국내 발생 환자는 지역별로 서울 7명, 경기 8명 등 수도권에서 15명이 확인됐고, 광주에서도 5명, 대전에서도 1명이 확진됐습니다.",0.835306,-0.549785
또한 코로나19로 1명이 추가 사망해 누적 사망자는 모두 289명으로 늘었습니다.,0.817843,0.575441


In [31]:
s

,"중앙방역대책본부는 오늘 0시 기준 코로나19 확진 환자가 44명 추가 확인돼 모두 13,417명으로 늘었다고 밝혔습니다.","신규 확진 환자 가운데 해외 유입 사례는 23명, 국내 발생은 21명입니다.","국내 발생 환자는 지역별로 서울 7명, 경기 8명 등 수도권에서 15명이 확인됐고, 광주에서도 5명, 대전에서도 1명이 확진됐습니다.",또한 코로나19로 1명이 추가 사망해 누적 사망자는 모두 289명으로 늘었습니다.
"중앙방역대책본부는 오늘 0시 기준 코로나19 확진 환자가 44명 추가 확인돼 모두 13,417명으로 늘었다고 밝혔습니다.",1.000000,0.702216,-0.040675,0.914619
"신규 확진 환자 가운데 해외 유입 사례는 23명, 국내 발생은 21명입니다.",0.702216,1.000000,0.682812,0.930119
"국내 발생 환자는 지역별로 서울 7명, 경기 8명 등 수도권에서 15명이 확인됐고, 광주에서도 5명, 대전에서도 1명이 확진됐습니다.",-0.040675,0.682812,1.000000,0.366781
또한 코로나19로 1명이 추가 사망해 누적 사망자는 모두 289명으로 늘었습니다.,0.914619,0.930119,0.366781,1.000000


## LSA 실습한거 뜯어보기

In [ ]:
# 함수 사용하지 않고 코드 써보고, 변수 각각 확인해보기

## Sklearn dataset *fetch_20newsgroups* 를 활용한 실습